In [1]:
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from random import shuffle
from mtcnn.mtcnn import MTCNN
from tensorflow.keras.models import load_model

In [2]:
# Constants:

img_size = 160
Data_DIR = "C:\\Users\\USER\\Desktop\\ML-Projects\\Face_Recognition\\Data" # Change to your directory
Project_DIR = "C:\\Users\\USER\\Desktop\\ML-Projects\\Face_Recognition" # Change to your directory
haarcascade_path = os.path.join(Project_DIR, "haarcascade_frontalface_default.xml")
dlib_detector_path = os.path.join(Project_DIR, "mmod_human_face_detector.dat")
facenet_path = os.path.join(Project_DIR, "facenet_keras.h5")
database_path = os.path.join(Project_DIR, "database.pickle")

In [3]:
faceCascade = cv2.CascadeClassifier(haarcascade_path)
detector = MTCNN()
face_net = load_model(facenet_path)
database = {}
cropped_faces = []

In [4]:
def get_embedding(face):
    face = cv2.resize(face, (img_size, img_size))
    face = face.astype('float32')
    mean, std = face.mean(), face.std()
    face = (face - mean) / std
    sample = np.expand_dims(face, axis=0)
    embeddings = face_net.predict(sample)
    return embeddings[0]

def add_image_to_database(path, name):
    img_array = cv2.imread(path)
    faces = detector.detect_faces(img_array)
    if len(faces) > 0:
        x, y, w, h = faces[0]['box'] # expects atmost one face in the image
        img_array = img_array[y : y + h, x : x + w]         
        img_array = cv2.resize(img_array, (img_size, img_size))
        cropped_faces.append(img_array)
        embedding = get_embedding(img_array)
        database[name] = embedding

def create_database():
    for img in os.listdir(Data_DIR):
        name = os.path.splitext(img)[0]
        path = os.path.join(Data_DIR, img)
        add_image_to_database(path, name)

In [33]:
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR) # supresses tf warnings
create_database()

In [7]:
print(len(database))
# print(len(cropped_faces))
# for face in cropped_faces:
#     plt.imshow(cv2.cvtColor(face, cv2.COLOR_BGR2RGB))
#     plt.show()

15


In [8]:
import pickle

# Save database:
with open(database_path, 'wb') as handle:
    pickle.dump(database, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [5]:
import pickle

# Load database:

# print(len(database))
# database = {}
# print(len(database))
with open(database_path, 'rb') as handle:
    database = pickle.load(handle)
    
print(len(database))

14


In [9]:
path = os.path.join(Project_DIR, "haarcascade_frontalface_default.xml")
faceCascade = cv2.CascadeClassifier(path)
font_scale = 1.5
font = cv2.FONT_HERSHEY_PLAIN

# for strictness, valid_distance could be kept closer to 10. If the system is allowed to be lenient, it can be in btw 15 to 20
valid_distance = 15  # Prediction is valid only if distance is less than or equal to valid_distance

cap = cv2.VideoCapture(0)

# Face detection with haarcascade (faster):
                       
while True:
    ret, frame = cap.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = faceCascade.detectMultiScale(gray, 1.1, 4)
    for x, y, w, h in faces:
        cv2.rectangle(frame, (x, y), (x + w, y + h), (200, 0, 0), 3)
        embedding = get_embedding(frame[y : y + h, x : x + w])
        min_dist = 1e5
        name = "Person"
        for person, embed in database.items():
            L2 = np.linalg.norm(embed - embedding)
            if(L2 < min_dist):
                min_dist = L2
                name = person
    
        if min_dist > valid_distance:
            name = "Unidentified"
        cv2.putText(frame, name, (x, y - 10), font, font_scale, (255, 255, 255), 2)
        
    cv2.imshow("Face Recognition", frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

        
# Face detection with mtcnn (slower):

# while True:
#     ret, frame = cap.read()
#     faces = detector.detect_faces(frame)
#     for face in faces:
#         x, y, w, h = face['box']
#         cv2.rectangle(frame, (x, y), (x + w, y + h), (200, 0, 0), 3)
#         cv2.putText(frame, "Person", (x, y - 10), font, font_scale, (255, 255, 255), 2)

#     cv2.imshow("Face Recognition", frame)
    
#     if cv2.waitKey(1) & 0xFF == ord('q'):
#         break
        
cap.release()
cv2.destroyAllWindows()

In [7]:
# In case webcam doesn't close properly, run this cell:
cap.release()
cv2.destroyAllWindows()